<a href="https://colab.research.google.com/github/parkmina365/deep_learning_with_python/blob/main/CH4_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D%EC%9D%98%20%EA%B8%B0%EB%B3%B8%EC%9A%94%EC%86%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4장. 머신러닝의 기본 요소

## 4.1 머신러닝의 네 가지 분류
4.1.1 지도학습
4.1.2 비지도학습
4.1.3 자기 지도 학습
- Autoencoder
4.1.4 강화 학습
- 알파고

## 4.2 머신 러닝 모델 평가
4.2.1 훈련, 검증, 테스트 세트\
4.2.2 기억해야 할 것
- 대표성 있는 데이터
- 시간의 방향
- 데이터 중복 

## 4.3 데이터 전처리, 특성 공학, 특성 학습
4.3.1 신경망을 위한 데이터 전처리
- 벡터화
- 값 정규화
- 누락된 값 다루기
- 특성 추출

4.3.2 특성 공학(Feature Engineering)

## 4.4 과대적합과 과소적합
- 과대적합 방지 방법
  - 훈련 데이터를 더 모음
  - 네트워크의 용량을 감소시킴
  - 가중치 규제를 추가
  - 드롭아웃 추가

4.4.1 네트워크 크기 축소

In [ ]:
from tensorflow.keras.datasets import imdb
import numpy as np

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

def vectorize_sequences(sequences, dimension=10000):
    # 크기가 (len(sequences), dimension))이고 모든 원소가 0인 행렬을 만듭니다
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.  # results[i]에서 특정 인덱스의 위치를 1로 만듭니다
    return results

# 훈련 데이터를 벡터로 변환합니다
x_train = vectorize_sequences(train_data)
# 테스트 데이터를 벡터로 변환합니다
x_test = vectorize_sequences(test_data)
# 레이블을 벡터로 변환합니다
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

In [ ]:
# 원본 모델
from keras import models
from keras import layers

original_model = models.Sequential()
original_model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
original_model.add(layers.Dense(16, activation='relu'))
original_model.add(layers.Dense(1, activation='sigmoid'))

original_model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# 작은 용량의 모델
smaller_model = models.Sequential()
smaller_model.add(layers.Dense(6, activation='relu', input_shape=(10000,)))
smaller_model.add(layers.Dense(6, activation='relu'))
smaller_model.add(layers.Dense(1, activation='sigmoid'))

smaller_model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
original_hist = original_model.fit(x_train, y_train,
                                   epochs=20, batch_size=512,
                                   validation_data=(x_test, y_test))

In [ ]:
smaller_model_hist = smaller_model.fit(x_train, y_train,
                                   epochs=20, batch_size=512,
                                   validation_data=(x_test, y_test))

In [ ]:
epochs=range(1,21)
original_val_loss = original_hist.history['val_loss']
smaller_val_loss = smaller_model_hist.history['val_loss']

In [ ]:
import matplotlib.pyplot as plt
plt.plot(epochs, original_val_loss, 'b+', label='Original model')
plt.plot(epochs, smaller_val_loss, 'bo', label='Small model')
plt.xlabel('Epochs')
plt.ylabel('Validation loss')
plt.legend()
plt.show()

In [ ]:
# 큰 용량의 모델
bigger_model = models.Sequential()
bigger_model.add(layers.Dense(1024, activation='relu', input_shape=(10000,)))
bigger_model.add(layers.Dense(1024, activation='relu'))
bigger_model.add(layers.Dense(1, activation='sigmoid'))

bigger_model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
bigger_model_hist = bigger_model.fit(x_train, y_train,
                                   epochs=20, batch_size=512,
                                   validation_data=(x_test, y_test))
epochs=range(1,21)
original_val_loss = original_hist.history['val_loss']
bigger_val_loss = bigger_model_hist.history['val_loss']

In [ ]:
plt.plot(epochs, original_val_loss, 'b+', label='Original model')
plt.plot(epochs, bigger_val_loss, 'bo', label='Big model')
plt.xlabel('Epochs')
plt.ylabel('Validation loss')
plt.legend()
plt.show()

### 용량이 큰 네트워크는 첫번째 에포크 이후 바로 과대적합 시작. 검증손실도 불안정

In [ ]:
original_train_loss = original_hist.history['loss']
bigger_model_train_loss = bigger_model_hist.history['loss']

plt.plot(epochs, original_train_loss, 'b+', label='Original model')
plt.plot(epochs, bigger_model_train_loss, 'bo', label='Bigger model')
plt.xlabel('Epochs')
plt.ylabel('Traning loss')
plt.legend()
plt.show()

### 용량이 큰 네트워크는 훈련 손실이 매우 빠르게 0에 가까워짐
### 용량이 많은 네트워크일수록 더 빠르게 훈련 데이터를 모델링 할 수 있음(훈련 손실이 낮아짐)
### 그러나 과대 적합에 민감해짐(훈련과 검증 손실 사이에 큰 차이가 발생함)

4.4.2 가중치 규제 추가

In [ ]:
from tensorflow.keras import regularizers

l2_model = models.Sequential()
l2_model.add(layers.Dense(16,kernel_regularizer=regularizers.l2(0.001),
                          activation='relu', input_shape=(10000,)))
l2_model.add(layers.Dense(16,kernel_regularizer=regularizers.l2(0.001),
                          activation='relu'))
l2_model.add(layers.Dense(1,activation='sigmoid'))

In [ ]:
l2_model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
l2_model_hist = l2_model.fit(x_train, y_train,
                             epochs=20, batch_size=512,
                             validation_data=(x_test, y_test))

In [ ]:
l2_model_val_loss = l2_model_hist.history['val_loss']

plt.plot(epochs, original_val_loss, 'b+', label='Original model')
plt.plot(epochs, l2_model_val_loss, 'bo', label='L2-regularized model')
plt.xlabel('Epochs')
plt.ylabel('Validation loss')
plt.legend()
plt.show()

### 두 모델이 동일한 파라미터수를 가지고 있더라도 L2규제를 사용한 모델이 기본 모델보다 과대적합에 잘 견디고 있음

In [ ]:
### l1(0.001) 모델과 비교해보기

4.4.3 드롭아웃 추가

In [ ]:
dpt_model = models.Sequential()
dpt_model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
dpt_model.add(layers.Dropout(0.5))
dpt_model.add(layers.Dense(16, activation='relu'))
dpt_model.add(layers.Dropout(0.5))
dpt_model.add(layers.Dense(1, activation='sigmoid'))

dpt_model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
dpt_model_hist = dpt_model.fit(x_train, y_train, epochs=20,
                               batch_size=512,
                               validation_data=(x_test, y_test))

In [ ]:
dpt_model_val_loss = dpt_model_hist.history['val_loss']

plt.plot(epochs, original_val_loss, 'b+', label='Original model')
plt.plot(epochs, dpt_model_val_loss, 'bo', label='Dropout-regularized model')
plt.xlabel('Epochs')
plt.ylabel('Validation loss')
plt.legend()
plt.show()

## 4.5 보편적인 머신 러닝 작업 흐름